# 0. Install and Import Dependencies

In [1]:
!pip install mediapipe opencv-python

In [2]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils #visualizing utilities
mp_pose = mp.solutions.pose #https://google.github.io/mediapipe/solutions/solutions.html

# 1. Make Detections

# 2. Determining Joints

<img src="https://i.imgur.com/0t2e6zy.png" style="height:300px">

# 3. Calculate Angles

In [3]:
def calc_angle(a,b,c):
    a = np.array(a) #11
    b = np.array(b) #13
    c = np.array(c) #15
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
    return angle

In [4]:
def visualizeText(image, angle, position):
    green = (124, 252, 0)
    red = (0,0,255)
    yellow = (0,255,255)
    
    if angle < 20.0 or angle > 170.0:
        cv2.putText(image, str("{:.2f}".format(angle)), 
                            tuple(np.multiply(position, [1280, 720]).astype(int)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, red, 2, cv2.LINE_AA
                       )
    elif angle >= 20.0 and angle <= 30.0:
        cv2.putText(image, str("{:.2f}".format(angle)), 
                            tuple(np.multiply(position, [1280, 720]).astype(int)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, yellow, 2, cv2.LINE_AA
                       )
    elif angle >= 160.0 and angle <= 170.0:
        cv2.putText(image, str("{:.2f}".format(angle)), 
                            tuple(np.multiply(position, [1280, 720]).astype(int)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, yellow, 2, cv2.LINE_AA
                       )
    else:
        cv2.putText(image, str("{:.2f}".format(angle)), 
                            tuple(np.multiply(position, [1280, 720]).astype(int)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, green, 2, cv2.LINE_AA
                       )

# 4. Curl Counter

In [5]:
cap = cv2.VideoCapture(0) #accessing the web cam: (zero of the video capture is the index of web cam)

#curl counter
counterLeft = 0
counterRight = 0
stageLeft = None
stageRight = None

#SETUP MEDIAPIPE INSTANCE
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened(): # loop
        ret, frame = cap.read()# video capture result
        
        #recolor bgr to rgb
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        #detection
        results = pose.process(image)
        #recolor rgb to bgr
        image.flags.writeable = True
        
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        #extract the landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            #calc the angle
            left_angle = calc_angle(left_shoulder, left_elbow, left_wrist)
            right_angle = calc_angle(right_shoulder, right_elbow, right_wrist)
           
            #visualize the text
            visualizeText(image, left_angle, left_elbow)
            visualizeText(image, right_angle, right_elbow)
            
            # update the stage
            if left_angle > 160:
                stageLeft ="down"
            if left_angle < 70 and stageLeft =='down':
                stageLeft ="up"
                counterLeft += 1
                
            if right_angle > 160:
                stageRight ="down"
            if right_angle < 70 and stageRight =='down':
                stageRight ="up"
                counterRight += 1
        except:
            pass
        
        #setup status box
        windowWidth=frame.shape[1]
        windowHeight=frame.shape[0]
        
        #background dashboard
        cv2.rectangle(image, (0,0),(windowWidth,int(windowHeight/10)),(242, 242, 242), -1)
        
        
        #show the text: left
        cv2.putText(image, 'COUNTS(right):', (15,45), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counterRight), (130,50), cv2.FONT_HERSHEY_SIMPLEX, 1.6, (74, 163, 208), 1, cv2.LINE_AA)
        cv2.putText(image, 'STAGE:', (200,45), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stageRight, (260,48), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (74, 163, 208), 1, cv2.LINE_AA)
        
        #show the text: right
        cv2.putText(image, 'COUNTS(left):', (int(windowWidth/2+275+15),45), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counterLeft), (int(windowWidth/2+275+130),50), cv2.FONT_HERSHEY_SIMPLEX, 1.6, (74, 163, 208), 1, cv2.LINE_AA)
        
        cv2.putText(image, 'STAGE', (int(windowWidth/2+275+200),45), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stageLeft, (int(windowWidth/2+275+260),48), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (74, 163, 208), 1, cv2.LINE_AA)
      
        #rendering detections (pts and connections)
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(74, 163, 208),thickness=2, circle_radius=2),
                                 mp_drawing.DrawingSpec(color=(74, 163, 208),thickness=2, circle_radius=2)
                                 )#image, pts, lines
        
        #imshow visualize the result(frame) as a popup window named 'mediapipe feed'
        cv2.imshow('Dumbbell Curls Coach', image)

        if cv2.waitKey(10) & 0xFf == ord('q'): #terminate video capture (hitting key Q)
            break


cv2.destroyWindow('Dumbbell Curls Coach')
cv2.waitKey(1)

cap.release() # releasing the web cam

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
